In [ ]:
from natasha import MorphVocab

morph_vocab = MorphVocab()

def lemmatization(token):
    forms = morph_vocab(token)
    forms_lst = []
    for f in forms:
        forms_lst.append(list(f)[2])
    forms_lst = list(set(forms_lst))
    return forms_lst

In [ ]:
def get_posts_for_token(qua: str):
    x = (qua, )
    
    list_id_posts = []
    for row in c.execute("""SELECT list_posts_id FROM tokens
                            WHERE token_str=?""", x):
        list_id_posts.append(row[-1])
    
    channel_names = []
    post_texts = []
    for id_p in list_id_posts:
        z = (id_p, )
        for row in c.execute("""SELECT channel_name, post_text FROM posts JOIN channels
            ON post.id_channel = channels.id WHERE id_post=?""", z):
            channel_names.append(row[-2])
            post_texts.append(post_text)
    return post_texts, channel_names

In [ ]:
def search_lemma(q):
    x = (q, )
    for row in c.execute("""SELECT id_token, list_id_tokens FROM tokens JOIN lemmas
        ON lemmas.id_lemma = tokens.id_lemma WHERE token_str=?""", x):
        tokens_id = list(row[-1])
        i = row[-2]
        tokens_id.remove(i)

    list_tokens = []
    for j in tokens_id:
        y = (j, )
        for row in c.execute("""SELECT token_str FROM tokens
                                WHERE id_token=?""", y):
            list_tokens.append(row[-1])

    post_texts = []
    channel_names = []
    for t in list_tokens:
        a, b = get_posts_for_token(t)
        post_texts.extend(a)
        channel_names.extend(b)
    return post_texts, channel_names

In [ ]:
def search_synonyms(q):
    x = (q, )
    for row in c.execute("""SELECT list_synonyms FROM tokens
        WHERE token_str=?""", x):
        synonyms_id = row[-1]

    list_synonyms = []
    for j in synonyms_id:
        y = (j, )
        for row in c.execute("""SELECT token_str FROM tokens
                                WHERE id_token=?""", y):
            list_synonyms.append(row[-1])
    
    for s in list_synonyms:
        a, b = get_posts_for_token(t)
        post_texts.extend(a)
        channel_names.extend(b)
    return post_texts, channel_names

In [ ]:
def search(qua: str):
    q = qua.lower()
    post_texts, channel_names = get_posts_for_token(q)
    a, b = search_lemma(q)
    post_texts.extend(a)
    channel_names.extend(b)
    c, d = search_synonyms(q)
    post_texts.extend(c)
    channel_names.extend(d)
    df_output = pd.DataFrame()
    df_output['post'] = post_texts
    df_output['channel'] = channel_names
    output = df_output.to_dict('records')
    with open('posts.json', 'a', encoding='utf-8') as file:
        json.dump(output, file)